In [1]:
from appgeopy import *
from my_packages import *

In [2]:
gps_stations = gpd.read_file(
    r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2024_Combined\Geodatabase\extract_inc_angle_20240805_TWD97_V3.shp",
    engine="pyogrio",
    read_geometry=False,
)
gps_stations.head(5)

,STATION,LONG,LAT,INC_ANGLE,Predicted,X_TWD97,Y_TWD97
0,ALIS,120.813300,23.508200,43.767014,43.766606,230933.332644,2.600572e+06
1,ANES,120.249069,23.703886,34.527428,34.525623,173423.766349,2.622432e+06
2,C001,120.612400,23.417900,40.414787,40.420660,210389.401261,2.590613e+06
3,C002,120.577200,23.361700,39.498070,39.501393,206773.878699,2.584399e+06
4,CAOT,120.688700,23.979400,43.841118,43.839385,218322.695324,2.652777e+06


In [3]:
fpath = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2024_Combined\5__Differencing\InSAR_ModelGPS_Differencing_NEU_v1.xlsx"

savefolder = os.path.dirname(fpath)
savename = "InSAR_ModelGPS_Differencing_LOS_v1.xlsx"
savepath = os.path.join(savefolder, savename)

available_sheetnames = data_io.get_sheetnames(fpath)

# select_sheet = available_sheetnames[0]
cache = []
for select_sheet in tqdm(available_sheetnames):

    if "_" in select_sheet:
        station_name = select_sheet.split("_")[0]
    else:
        station_name = select_sheet

    select_gps = gps_stations.query("STATION==@station_name")

    if len(select_gps) > 0:
        inc_angle = select_gps.INC_ANGLE.values[0]

        df = pd.read_excel(
            fpath,
            parse_dates=[0],
            usecols=[0, 4, 5, 6],
            index_col=[0],
            sheet_name=select_sheet,
        )

        LOS_disp = df.apply(
            lambda row: insartools.get_LOS_disp(
                *row, incidence_angle=inc_angle
            ),
            axis=1,
        )

        df["model_LOS(mm)"] = LOS_disp
        df = df.reset_index(drop=False)
        data_io.save_df_to_excel(
            df_to_save=df,
            filepath=savepath,
            sheet_name=select_sheet,
            verbose=False,
        )
    else:
        cache.append(select_sheet)

100%|████████████████████████████████████████████████████████████████████████████████| 108/108 [01:33<00:00,  1.15it/s]
